In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
# import tushare as ts
import QUANTAXIS as QA
import talib as ta
import datetime

In [60]:
# from EmQuantAPI import *
c.start()
# c.stop()

[EmQuantAPI Python] [Em_Info][2018-11-02 21:49:15]:The current version is EmQuantAPI(V2.3.2.0).

[EmQuantAPI Python] [Em_Info][2018-11-02 21:49:15]:connect server...

[EmQuantAPI Python] [Em_Info][2018-11-02 21:49:15]:verifying your token...

[EmQuantAPI Python] [Em_Info][2018-11-02 21:49:16]:start success!



ErrorCode=0, ErrorMsg=success, Data={}

In [3]:
df_basic = pd.read_excel('data/ipo_info.xlsx', header=0, dtype={'code':str})
codes = df_basic.code_choice.tolist()

In [61]:
# sample
import datetime
code = codes[0]
start = "2018-11-02"
end = "2018-11-02"

# c.csd(code,"OPEN,CLOSE,HIGH,LOW,VOLUME",start,end,
# "period=1,adjustflag=1,curtype=1,pricetype=1,order=1,market=CNSESH,Ispandas=1")

In [62]:
c.csd(code,"OPEN,CLOSE,HIGH,LOW,VOLUME",start,end,
"period=1,adjustflag=1,curtype=1,pricetype=1,order=1,market=CNSESH,Ispandas=1")

,DATES,OPEN,CLOSE,HIGH,LOW,VOLUME
CODES,,,,,,
000001.SZ,2018/11/02,11.04,11.09,11.16,10.83,221292287


In [63]:
df_daily = c.csd(code,"OPEN,CLOSE,HIGH,LOW,VOLUME",start,end,
"period=1,adjustflag=1,curtype=1,pricetype=1,order=1,market=CNSESH,Ispandas=1")
df_daily = df_daily.reset_index()
df_daily.columns = ['code', 'date', 'open', 'close', 'high', 'low', 'volume']
df_daily.code = df_daily.code.apply(lambda x : x[:6])
df_daily.date = df_daily.date.apply(lambda x : datetime.datetime.strptime(
                                    x, '%Y/%m/%d').strftime('%Y-%m-%d'))

In [74]:
df_daily

,code,date,open,close,high,low,volume
0,000001,2018-11-02,11.04,11.09,11.16,10.83,221292287


In [18]:
start = '2015-01-01'
end = '2018-09-30'

In [19]:
df_basic['_start'] = df_basic.timeToMarket.apply(lambda x : max(x, start))

In [20]:
df_basic['_end'] = df_basic.timeToMarket.apply(lambda x : max(x, end))

In [29]:
df_basic = df_basic.set_index('code')

In [ ]:
QA.QA_util_get_trade_gap()

In [32]:
df_basic.head()

,name,issueprice,timeToMarket,code_choice,_start,_end
code,,,,,,
000001,平安银行,40.0,1991-04-03,000001.SZ,2015-01-01,2018-09-30
000002,万科A,1.0,1991-01-29,000002.SZ,2015-01-01,2018-09-30
000004,国农科技,1.0,1991-01-14,000004.SZ,2015-01-01,2018-09-30
000005,世纪星源,10.0,1990-12-10,000005.SZ,2015-01-01,2018-09-30
000006,深振业A,10.0,1992-04-27,000006.SZ,2015-01-01,2018-09-30


In [34]:
days = pd.Series()
for i in df_basic.index.tolist():
    days.loc[i] = QA.QA_util_get_trade_gap(df_basic.loc[i, '_start'],
                                         df_basic.loc[i, '_end'])    

In [35]:
days.head()

000001    915
000002    915
000004    915
000005    915
000006    915
dtype: int64

In [37]:
df_basic['trade_days'] = days

In [59]:
writer = pd.ExcelWriter('data/stock_basic_info.xlsx')
df_basic.to_excel(writer)
writer.save()

In [45]:
df_basic.head()

,name,issueprice,timeToMarket,code_choice,_start,_end,trade_days
code,,,,,,,
000001,平安银行,40.0,1991-04-03,000001.SZ,2015-01-01,2018-09-30,915
000002,万科A,1.0,1991-01-29,000002.SZ,2015-01-01,2018-09-30,915
000004,国农科技,1.0,1991-01-14,000004.SZ,2015-01-01,2018-09-30,915
000005,世纪星源,10.0,1990-12-10,000005.SZ,2015-01-01,2018-09-30,915
000006,深振业A,10.0,1992-04-27,000006.SZ,2015-01-01,2018-09-30,915


In [43]:
df_rec = pd.read_excel('data/record.xlsx', header=0, dtype={'code':str})
df_rec = df_rec.set_index('code')

In [46]:
df_rec

,records
code,
000001,915
000300,915
399001,915
399005,915
399006,915
300126,826
600319,657
000633,564
000711,513


In [55]:
df_diff = pd.Series()
for i in df_basic.index.tolist():
    try:
        df_diff.loc[i] = (df_basic.loc[i, 'trade_days'] - df_rec.loc[i])[0]
    except:
        print(i)

002939
002940
300694
300760
601162


In [56]:
df_diff

000001      0
000002    135
000004    173
000005     97
000006    117
000007    493
000008    153
000009     32
000010     99
000011      0
000012     24
000014      0
000016     85
000017     32
000018     25
000019    157
000020    147
000021     20
000022    276
000023    188
000025     16
000026      7
000027      0
000028    111
000029    498
000030      7
000031    181
000032    143
000034    435
000035    330
         ... 
603936     98
603937      0
603938      0
603939    110
603955     40
603958      0
603959      4
603960     55
603963      0
603966      0
603968      0
603969      0
603970      0
603976      0
603977     78
603978      3
603979      0
603980      1
603985      0
603986    200
603987      0
603988     14
603989      3
603990      0
603991      0
603993     19
603996      0
603997     91
603998     76
603999      3
Length: 3551, dtype: int64

In [57]:
df_basic['diff'] = df_diff

In [58]:
df_basic.head()

,name,issueprice,timeToMarket,code_choice,_start,_end,trade_days,diff
code,,,,,,,,
000001,平安银行,40.0,1991-04-03,000001.SZ,2015-01-01,2018-09-30,915,0.0
000002,万科A,1.0,1991-01-29,000002.SZ,2015-01-01,2018-09-30,915,135.0
000004,国农科技,1.0,1991-01-14,000004.SZ,2015-01-01,2018-09-30,915,173.0
000005,世纪星源,10.0,1990-12-10,000005.SZ,2015-01-01,2018-09-30,915,97.0
000006,深振业A,10.0,1992-04-27,000006.SZ,2015-01-01,2018-09-30,915,117.0
